In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# Importación de archivos publicos en mega

!apt-get install megatools

!megadl 'https://mega.nz/file/IYR13BIL#xJGXa9RVya2lCZa97LnMcuyc-DJ8eSg_QrL9LwAPeGc'
!megadl 'https://mega.nz/file/sBYlFT5S#dyje3tq74m--QAOs9B60zR-f_9yBnUT54lYfjNZ2Rus'

# Lectura archivos
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
megatools is already the newest version (1.10.3-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

ERROR: Download failed for 'https://mega.nz/file/IYR13BIL#xJGXa9RVya2lCZa97LnMcuyc-DJ8eSg_QrL9LwAPeGc': Can't rename donwloaded temporary file /content/.megatmp.IYR13BIL to /content/train.csv (downloaded data are good!): Error moving file /content/.megatmp.IYR13BIL: File exists
Downloaded test.csv


##Preprocesado

In [ ]:
df.head(5)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


#Rellenar datos nulos

In [ ]:
#Eliminar filas con demasiados datos nulos
num_nulos_por_fila = df.isnull().sum(axis=1)
df = df[num_nulos_por_fila <= 3]

In [ ]:
print(df.isnull().sum())

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     4585
ESTU_HORASSEMANATRABAJA            9194
FAMI_ESTRATOVIVIENDA              10309
FAMI_TIENEINTERNET                 4833
FAMI_EDUCACIONPADRE                2031
FAMI_EDUCACIONMADRE                1886
ESTU_PAGOMATRICULAPROPIO           4788
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [ ]:
print(df['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts())

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      198589
Entre 2.5 millones y menos de 4 millones    123830
Menos de 500 mil                             77035
Entre 500 mil y menos de 1 millón            76348
Entre 4 millones y menos de 5.5 millones     67766
Más de 7 millones                            66193
Entre 5.5 millones y menos de 7 millones     37341
No pagó matrícula                            18940
Name: count, dtype: int64


In [ ]:
# Cambio valores nulos en matricula a No pago matricula
df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("No pagó matrícula")

In [ ]:
print(df['ESTU_HORASSEMANATRABAJA'].value_counts())

ESTU_HORASSEMANATRABAJA
Más de 30 horas        249277
0                      116518
Entre 11 y 20 horas    115821
Entre 21 y 30 horas     92659
Menos de 10 horas       87158
Name: count, dtype: int64


In [ ]:
# Cambio valores nulos a el más común Más de 30 horas
df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna('Más de 30 horas')

In [ ]:
print(df['FAMI_ESTRATOVIVIENDA'].value_counts())

FAMI_ESTRATOVIVIENDA
Estrato 2      232660
Estrato 3      210669
Estrato 1      111984
Estrato 4       65509
Estrato 5       23605
Estrato 6       12604
Sin Estrato      3287
Name: count, dtype: int64


In [ ]:
# Cambio valores nulos a el más común Estrato 2
df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')

In [ ]:
print(df['FAMI_TIENEINTERNET'].value_counts())

FAMI_TIENEINTERNET
Si    592450
No     73344
Name: count, dtype: int64


In [ ]:
# Cambio valores nulos a el más común Si
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].fillna('Si')

In [ ]:
print(df['FAMI_EDUCACIONPADRE'].value_counts())

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      128180
Primaria incompleta                     125560
Educación profesional completa           83011
Secundaria (Bachillerato) incompleta     71591
Técnica o tecnológica completa           62938
Primaria completa                        55890
Postgrado                                44122
Educación profesional incompleta         27028
Técnica o tecnológica incompleta         22518
Ninguno                                  21956
No sabe                                  16580
No Aplica                                 9222
Name: count, dtype: int64


In [ ]:
# Cambio valores nulos a no aplica
df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna('No Aplica')

In [ ]:
print(df['FAMI_EDUCACIONMADRE'].value_counts())

FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      141726
Primaria incompleta                      99404
Técnica o tecnológica completa           89532
Educación profesional completa           85317
Secundaria (Bachillerato) incompleta     81004
Primaria completa                        56119
Postgrado                                46234
Técnica o tecnológica incompleta         27529
Educación profesional incompleta         22465
Ninguno                                  14476
No sabe                                   3017
No Aplica                                 1918
Name: count, dtype: int64


In [ ]:
# Cambio valores nulos a no aplica
df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna('No Aplica')

In [ ]:
print(df['ESTU_PAGOMATRICULAPROPIO'].value_counts())

ESTU_PAGOMATRICULAPROPIO
No    370991
Si    294848
Name: count, dtype: int64


In [ ]:
# Cambio valores nulos a el más común No
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].fillna('No')

In [ ]:
print(df.isnull().sum())

ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


#Convertir a valores numericos las columnas de Si o No

In [ ]:
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].map({'Si': 1, 'No': 0})
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].map({'Si': 1, 'No': 0})

# Eliminar columnas irrelevantes

In [ ]:
df = df.drop(columns=['ID'])

# Variables Categoricas con label encoding

In [ ]:
label_encoder = LabelEncoder()
df['ESTU_PRGM_ACADEMICO'] = label_encoder.fit_transform(df['ESTU_PRGM_ACADEMICO'])
df['ESTU_PRGM_DEPARTAMENTO'] = label_encoder.fit_transform(df['ESTU_PRGM_DEPARTAMENTO'])
df['ESTU_VALORMATRICULAUNIVERSIDAD'] = label_encoder.fit_transform(df['ESTU_VALORMATRICULAUNIVERSIDAD'])
df['ESTU_HORASSEMANATRABAJA'] = label_encoder.fit_transform(df['ESTU_HORASSEMANATRABAJA'])
df['FAMI_EDUCACIONPADRE'] = label_encoder.fit_transform(df['FAMI_EDUCACIONPADRE'])
df['FAMI_EDUCACIONMADRE'] = label_encoder.fit_transform(df['FAMI_EDUCACIONMADRE'])
df['FAMI_ESTRATOVIVIENDA'] = label_encoder.fit_transform(df['FAMI_ESTRATOVIVIENDA'])
df['RENDIMIENTO_GLOBAL'] = label_encoder.fit_transform(df['RENDIMIENTO_GLOBAL'])


In [ ]:
df.head(15)

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,20212,300,4,3,3,2,1,11,5,0,2
1,20212,249,3,1,0,2,0,10,11,0,1
2,20203,818,4,1,4,2,1,8,8,0,1
3,20195,14,26,2,0,3,1,4,8,0,0
4,20212,905,1,1,2,2,1,6,6,0,3
5,20203,810,1,6,3,4,1,0,8,0,2
6,20183,455,16,1,2,1,1,1,10,1,0
7,20183,94,4,0,1,1,1,7,9,1,3
8,20212,452,3,3,3,0,1,8,1,1,3
9,20183,14,1,1,4,4,1,5,5,1,0


#Entrenamiento del Modelo XGBoost

In [ ]:
#Division de los datos
X_train = df.drop('RENDIMIENTO_GLOBAL', axis=1)
y_train = df['RENDIMIENTO_GLOBAL']

In [ ]:
#Entrenamiento del modelo con XGBoost
model = XGBClassifier(random_state=42, eval_metric='logloss')
#Hiperparametros posibles para aplicar al Grid Search
parameters = {
    'learning_rate': [1, 0.1],
    'n_estimators': [150,180],
    'max_depth': [5, 8, 10]
}

In [ ]:
# Tarda 40 minutos
#Aplicar Grid Search para hacer una busqueda exhaustiva de los mejores hiperparametros, Se utiliza validación cruzada = 5
grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [1, 0.1], 'max_depth': [5, 8, 10],
                         'n_estimators': [150, 180]},
             scoring='accuracy')

In [ ]:
#Imprimir todos los resultados que se encontraron en el Grid Search
cv_results = grid_search.cv_results_
for mean_score, std, params in zip(cv_results["mean_test_score"], cv_results["std_test_score"], cv_results["params"]):
    print(f"Precisión: {mean_score:.4f}, Desviación: {std:.4f} para los parámetros {params}" )

Precisión: 0.4338, Desviación: 0.0011 para los parámetros {'learning_rate': 1, 'max_depth': 5, 'n_estimators': 150}
Precisión: 0.4336, Desviación: 0.0016 para los parámetros {'learning_rate': 1, 'max_depth': 5, 'n_estimators': 180}
Precisión: 0.4175, Desviación: 0.0008 para los parámetros {'learning_rate': 1, 'max_depth': 8, 'n_estimators': 150}
Precisión: 0.4156, Desviación: 0.0011 para los parámetros {'learning_rate': 1, 'max_depth': 8, 'n_estimators': 180}
Precisión: 0.4015, Desviación: 0.0010 para los parámetros {'learning_rate': 1, 'max_depth': 10, 'n_estimators': 150}
Precisión: 0.3988, Desviación: 0.0007 para los parámetros {'learning_rate': 1, 'max_depth': 10, 'n_estimators': 180}
Precisión: 0.4253, Desviación: 0.0017 para los parámetros {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 150}
Precisión: 0.4273, Desviación: 0.0017 para los parámetros {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 180}
Precisión: 0.4339, Desviación: 0.0015 para los parámetros {'learni

In [ ]:
#Se escoge el que tenga mejor precisión menos desviación pero a su vez menos complejidad a vista del desarrollador:
#Precisión: 0.4338, Desviación: 0.0011 para los parámetros {'learning_rate': 1, 'max_depth': 5, 'n_estimators': 150}
xgb_model = XGBClassifier(learning_rate=1, max_depth=5, n_estimators=150, random_state=42)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
#Calcular Precisión
y_pred = xgb_model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.46
